In [5]:
input_movie = "yuki_dance_full_vid.mov"

In [6]:
import cv2
import numpy as np
import math

def measure_speed(video):
    count = 0
    
    vc = cv2.VideoCapture(video)
    
    total_frames = int(vc.get(cv2.CAP_PROP_FRAME_COUNT))
    fps = int(vc.get(cv2.CAP_PROP_FPS))
    duration = total_frames/fps # float

    width  = int(vc.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(vc.get(cv2.CAP_PROP_FRAME_HEIGHT))
    
    batches_diff = dict()
    batches_frame_range = dict()
    
    curr_frame = 0
    while vc.isOpened():
        batches_frame_range[count] = [curr_frame, curr_frame]
        
        pixel_diffs = np.zeros(shape=(height, width), dtype=float)
        prevFrame = None
        
        ret = True
        i = 0
        while i < fps and curr_frame < total_frames:
            ret, frame = vc.read()
            curr_frame += 1
            if not ret:
                break
                
            grayFrame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            if not i:
                prevFrame = grayFrame
                i += 1
                continue
            
        
            pixel_diffs = np.add(pixel_diffs, np.abs(grayFrame.astype(float) - prevFrame.astype(float)))
            prevFrame = grayFrame
            i += 1
        
        batches_frame_range[count][-1] = curr_frame - 1
        batches_diff[count] = np.sum(pixel_diffs)
        count += 1
            
        # video finished
        if not ret or curr_frame >= total_frames:
            vc.release()
            cv2.destroyAllWindows()
    return batches_frame_range, batches_diff

In [7]:
# COMPUTING NORMALIZED BATCH DIFFERENCES
batches_frame_range, batches_diff = measure_speed(input_movie)

min_diff = float('inf')
min_batch = float('inf')
for k, v in batches_diff.items():
    if min_diff > v:
        min_batch = k
        min_diff = v

normalized_batches_diff = batches_diff.copy()
for k in normalized_batches_diff.keys():
    normalized_batches_diff[k] /= min_diff
    

print(normalized_batches_diff)
# RUNNING MOVIE WITH SPEEDS ALTERED BY BATCH
vc = cv2.VideoCapture(input_movie)
output = cv2.VideoWriter('yuki_slowed_down.mp4', -1, 20.0, (568, 320))

avg_weight = sum(normalized_batches_diff) / len(normalized_batches_diff)
while vc.isOpened():
    curr_batch = 0
    while curr_batch < max(batches_frame_range.keys()) + 1:
        for _ in range(batches_frame_range[curr_batch][1] - batches_frame_range[curr_batch][0] + 1):
            ret, frame = vc.read()
            cv2.imshow("", frame)
            
            ## perhaps slow down worse ones even more??
            cv2.waitKey(int(normalized_batches_diff[curr_batch] * 1.5))

#             scaling_factor = np.log(normalized_batches_diff[curr_batch] / avg_weight) * 1.35
#             if (scaling_factor <= 1):
#                 scaling_factor = 1.65
#             elif (scaling_factor > 3.2):
#                 scaling_factor = 3.2
#             cv2.waitKey(int(normalized_batches_diff[curr_batch] * scaling_factor))
            
        curr_batch += 1
        
    vc.release()
    cv2.destroyAllWindows()

{0: 91.84881174834554, 1: 51.584677803590566, 2: 47.87484489821501, 3: 39.242760345511996, 4: 33.951932332562684, 5: 73.28767068839969, 6: 83.13117623588414, 7: 64.93036050877971, 8: 85.71964017914607, 9: 60.703577366936045, 10: 52.71776800187116, 11: 49.972431501701216, 12: 34.27116658454022, 13: 34.566899763098746, 14: 52.33660412317235, 15: 43.248556757189206, 16: 46.8261112167063, 17: 55.451425560570335, 18: 52.17253331471687, 19: 63.508750670727636, 20: 80.8896735206156, 21: 56.512561269982335, 22: 35.35930669311019, 23: 32.46411753645115, 24: 45.87363618713823, 25: 65.06570200787701, 26: 64.4600098551738, 27: 51.793490795298254, 28: 34.01427267645424, 29: 22.0299527980999, 30: 33.75926929033067, 31: 66.15581264162991, 32: 83.43375453075564, 33: 73.86310531736055, 34: 58.29127083332272, 35: 51.17386075668779, 36: 51.73082432362882, 37: 56.05730014099931, 38: 46.675760196206625, 39: 1.0}


In [3]:
yuki_weights = {0: 91.84881174834554, 1: 51.584677803590566, 2: 47.87484489821501, 3: 39.242760345511996, 4: 33.951932332562684, 5: 73.28767068839969, 6: 83.13117623588414, 7: 64.93036050877971, 8: 85.71964017914607, 9: 60.703577366936045, 10: 52.71776800187116, 11: 49.972431501701216, 12: 34.27116658454022, 13: 34.566899763098746, 14: 52.33660412317235, 15: 43.248556757189206, 16: 46.8261112167063, 17: 55.451425560570335, 18: 52.17253331471687, 19: 63.508750670727636, 20: 80.8896735206156, 21: 56.512561269982335, 22: 35.35930669311019, 23: 32.46411753645115, 24: 45.87363618713823, 25: 65.06570200787701, 26: 64.4600098551738, 27: 51.793490795298254, 28: 34.01427267645424, 29: 22.0299527980999, 30: 33.75926929033067, 31: 66.15581264162991, 32: 83.43375453075564, 33: 73.86310531736055, 34: 58.29127083332272, 35: 51.17386075668779, 36: 51.73082432362882, 37: 56.05730014099931, 38: 46.675760196206625, 39: 1.0}

In [7]:
yuki_weights = {0: 91.84881174834554, 1: 51.584677803590566, 2: 47.87484489821501, 3: 39.242760345511996, 4: 33.951932332562684, 5: 73.28767068839969, 6: 83.13117623588414, 7: 64.93036050877971, 8: 85.71964017914607, 9: 60.703577366936045, 10: 52.71776800187116, 11: 49.972431501701216, 12: 34.27116658454022, 13: 34.566899763098746, 14: 52.33660412317235, 15: 43.248556757189206, 16: 46.8261112167063, 17: 55.451425560570335, 18: 52.17253331471687, 19: 63.508750670727636, 20: 80.8896735206156, 21: 56.512561269982335, 22: 35.35930669311019, 23: 32.46411753645115, 24: 45.87363618713823, 25: 65.06570200787701, 26: 64.4600098551738, 27: 51.793490795298254, 28: 34.01427267645424, 29: 22.0299527980999, 30: 33.75926929033067, 31: 66.15581264162991, 32: 83.43375453075564, 33: 73.86310531736055, 34: 58.29127083332272, 35: 51.17386075668779, 36: 51.73082432362882, 37: 56.05730014099931, 38: 46.675760196206625, 39: 1.0}
min_weight = 46.675760196206625
for k in yuki_weights.keys():
    yuki_weights[k] /= 46.675760196206625
    yuki_weights[k] = np.e ** yuki_weights[k]

yuki_weights

{0: 7.154956790828264,
 1: 3.0197395861055276,
 2: 2.7890182892553494,
 3: 2.3181106130625144,
 4: 2.0696917125436425,
 5: 4.807341005621137,
 6: 5.935998736284932,
 7: 4.0192438890273845,
 8: 6.274486011772089,
 9: 3.6712695147770837,
 10: 3.093943131535318,
 11: 2.9172144154335022,
 12: 2.083895682556901,
 13: 2.0971409635707645,
 14: 3.068780236720131,
 15: 2.525841398000442,
 16: 2.727052020844245,
 17: 3.28055736253338,
 18: 3.0580120486475577,
 19: 3.8986746599321003,
 20: 5.65767171454228,
 21: 3.355992413051954,
 22: 2.133047720185176,
 23: 2.004759554703766,
 24: 2.671967191780689,
 25: 4.03091504135209,
 26: 3.9789454372436093,
 27: 3.0332792355472855,
 28: 2.0724578484202687,
 29: 1.6031628299787144,
 30: 2.061166275211461,
 31: 4.126164859869323,
 32: 5.974604184668954,
 33: 4.866974373663362,
 34: 3.4863498724763087,
 35: 2.9932779478888087,
 36: 3.0292095136960957,
 37: 3.3234182043154603,
 38: 2.718281828459045,
 39: 1.021655546827197}

In [ ]:
# real stuff

In [15]:
import cv2
import numpy as np

In [16]:
def measure_speed(video):
    count = 0
    
    vc = cv2.VideoCapture(video)
    
    total_frames = int(vc.get(cv2.CAP_PROP_FRAME_COUNT))
    fps = int(vc.get(cv2.CAP_PROP_FPS))
    duration = total_frames/fps # float
    
    width  = int(vc.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(vc.get(cv2.CAP_PROP_FRAME_HEIGHT))
    
    batches_diff = dict()
    batches_frame_range = dict()
    
    curr_frame = 0
    while vc.isOpened():
        batches_frame_range[count] = [curr_frame, curr_frame]
        
        pixel_diffs = np.zeros(shape=(height, width), dtype=float)
        prevFrame = None
        
        ret = True
        i = 0
        while i < fps and curr_frame < total_frames:
            ret, frame = vc.read()
            curr_frame += 1
            if not ret:
                break
                
            grayFrame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            if not i:
                prevFrame = grayFrame
                i += 1
                continue
            
        
            pixel_diffs = np.add(pixel_diffs, np.abs(grayFrame.astype(float) - prevFrame.astype(float)))
#             print("prevFrame", prevFrame[10][:20])
#             print("grayFrame", grayFrame[10][:20])
#             print("pixel_diffs", pixel_diffs[10][:20])

            prevFrame = grayFrame
            i += 1
        
        batches_frame_range[count][-1] = curr_frame - 1
        batches_diff[count] = np.sum(pixel_diffs)
        count += 1
            
        # video finished
        if not ret or curr_frame >= total_frames:
            vc.release()
            cv2.destroyAllWindows()
    
#     print(fps)
#     print(total_frames)
#     print("outside while loop!\n")
#     print(batches_diff.items())
#     print("\n", batches_frame_range.items())
    return batches_frame_range, batches_diff

In [17]:
batches_frame_range, batches_diff = measure_speed(input_movie)
print(batches_diff, "\n")

min_diff = float('inf')
min_batch = float('inf')
for k, v in batches_diff.items():
    if min_diff > v:
        min_batch = k
        min_diff = v

normalized_batches_diff = batches_diff.copy()
for k in normalized_batches_diff.keys():
    normalized_batches_diff[k] /= min_diff
    
print(normalized_batches_diff, "\n")

{0: 4667023.0, 1: 6368225.0, 2: 3158268.0, 3: 3371236.0, 4: 2858192.0, 5: 3043888.0, 6: 2081803.0, 7: 1562428.0, 8: 906721.0, 9: 2280888.0, 10: 1332244.0, 11: 409456.0} 

{0: 11.398106267828533, 1: 15.55289213004572, 2: 7.713326950881169, 3: 8.233451213317181, 4: 6.9804618811300845, 5: 7.433980696338557, 6: 5.08431430971826, 7: 3.815862998710484, 8: 2.214452834981048, 9: 5.570532609120394, 10: 3.253692704466414, 11: 1.0} 



In [ ]:
import math
vc = cv2.VideoCapture("highdef.mov")
output = cv2.VideoWriter('relative_slowdown.mp4', -1, 20.0, (568, 320))

while vc.isOpened():
    curr_batch = 0
    while curr_batch < max(batches_frame_range.keys()) + 1:
        for _ in range(batches_frame_range[curr_batch][1] - batches_frame_range[curr_batch][0] + 1):
            ret, frame = vc.read()
            cv2.imshow("", frame)
#             cv2.waitKey(math.ceil(1 * normalized_batches_diff[curr_batch]))
            cv2.waitKey(100)
            
        curr_batch += 1
        
    vc.release()
    cv2.destroyAllWindows()

In [ ]:
# trying code from here: https://www.learnopencv.com/video-stabilization-using-point-feature-matching-in-opencv/

In [3]:
# Import numpy and OpenCV
import numpy as np
import cv2

# Read input video
input_vid = "highdef.mov"
cap = cv2.VideoCapture(input_vid);

# Get frame count
n_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT)) 

# Get width and height of video stream
w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)) 
h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Define the codec for output video
fourcc = cv2.VideoWriter_fourcc(*'MJPG')

# Set up output video
output_vid = "video_out.mp4"
fps = cap.get(cv2.CAP_PROP_FPS)

out = cv2.VideoWriter(output_vid, fourcc, fps, (w, h))

In [4]:
# Read first frame
_, prev = cap.read() 

# Convert frame to grayscale
prev_gray = cv2.cvtColor(prev, cv2.COLOR_BGR2GRAY) 

In [12]:
# Pre-define transformation-store array
transforms = np.zeros((n_frames-1, 3), np.float32) 

for i in range(n_frames-2):
    # Detect feature points in previous frame
    prev_pts = cv2.goodFeaturesToTrack(prev_gray, \
                                     maxCorners=200, \
                                     qualityLevel=0.01, \
                                     minDistance=30, \
                                     blockSize=3)
    
    # Read next frame
    success, curr = cap.read() 
    if not success: 
        break 
    
    # Convert to grayscale
    curr_gray = cv2.cvtColor(curr, cv2.COLOR_BGR2GRAY) 
    
    # Calculate optical flow (i.e. track feature points)
    curr_pts, status, err = cv2.calcOpticalFlowPyrLK(prev_gray, curr_gray, prev_pts, None) 
    
    # Sanity check
    assert prev_pts.shape == curr_pts.shape 
    
    # Filter only valid points
    idx = np.where(status==1)[0]
    prev_pts = prev_pts[idx]
    curr_pts = curr_pts[idx]

    #Find transformation matrix
    m = cv2.estimateAffine2D(prev_pts, curr_pts) #will only work with OpenCV-3 or less
   
    # Extract traslation
    dx = m[0,2]
    dy = m[1,2]

    # Extract rotation angle
    da = np.arctan2(m[1,0], m[0,0])
   
    # Store transformation
    transforms[i] = [dx,dy,da]
   
    # Move to next frame
    prev_gray = curr_gray

    print("Frame: " + str(i) +  "/" + str(n_frames) + " -  Tracked points : " + str(len(prev_pts)))

TypeError: tuple indices must be integers or slices, not tuple

In [10]:
cv2.__version__

'4.1.1-openvino'